In [3]:
import qutip as qt
import numpy as np
from utils import __all__, pprint

In [2]:
__all__

['prettyprint',
 'pprint',
 'format_complex',
 'plot_bloch_plotly',
 'plot_bloch',
 '__version__']

In [275]:
I = qt.qeye(2)
X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()
H = (X+Z)/(np.sqrt(2)) # equivalent to (X+Z)/sqrt(2)

one = qt.basis(2, 1)
zero = qt.basis(2, 0)
plus = (one + zero).unit()
minus = (one - zero).unit()
psi_p = (qt.tensor(zero, one) + qt.tensor(one, zero)).unit()
psi_m = (qt.tensor(zero, one) - qt.tensor(one, zero)).unit()
phi_p = (qt.tensor(zero, zero) + qt.tensor(one, one)).unit()
phi_m = (qt.tensor(zero, zero) - qt.tensor(one, one)).unit()
phi_p.basis_expansion(), phi_m.basis_expansion(), psi_p.basis_expansion(), psi_m.basis_expansion()

('(0.70710678118655+0j) |11> + (0.70710678118655+0j) |00>',
 '(-0.70710678118655+0j) |11> + (0.70710678118655+0j) |00>',
 '(0.70710678118655+0j) |10> + (0.70710678118655+0j) |01>',
 '(-0.70710678118655+0j) |10> + (0.70710678118655+0j) |01>')

In [325]:
print(qt.tensor(X, I)*phi_p == psi_p)
print(qt.tensor(Z, I)*phi_p == phi_m)
print(qt.tensor(Z, I)*qt.tensor(X, I)*phi_p == psi_m)

True
True
True


In [277]:
test = phi_p.copy()
display(test)
print(test.basis_expansion())
test = qt.tensor(X,I)*test
display(test)
print(test.basis_expansion())
test = qt.tensor(Z,I)*test
display(test)
print(test.basis_expansion())

Quantum object: dims=[[2, 2], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.70710678]]

(0.70710678118655+0j) |11> + (0.70710678118655+0j) |00>


Quantum object: dims=[[2, 2], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.        ]
 [0.70710678]
 [0.70710678]
 [0.        ]]

(0.70710678118655+0j) |10> + (0.70710678118655+0j) |01>


Quantum object: dims=[[2, 2], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.        ]
 [ 0.70710678]
 [-0.70710678]
 [ 0.        ]]

(-0.70710678118655+0j) |10> + (0.70710678118655+0j) |01>


In [278]:
# Alice want to send bob two bits x and y.
# They share a Bell pair in state phi+ = (|00> + |11>)/sqrt(2).
# The first qubit is with Alice, the second with Bob.
# Alice's first bit is x, second bit is y 
# Protocol:
# Alice applies the following operations to her qubit:
# if x=1 she applies X to her qubit
# if y=1 she applies Z to her qubit
# she sends her qubit to Bob
# Bob applies CNOT and H to his qubit
# he measures both qubits in the computational basis
# he gets x and y

CNOT = qt.gates.cnot()
CNOT12 = (qt.tensor(I,I) + qt.tensor(Z,I) + qt.tensor(I,X) - qt.tensor(Z,X))*0.5
CNOT21 =  (qt.tensor(I,I) + qt.tensor(I, Z) + qt.tensor(X,I) - qt.tensor(X,Z))*0.5
IH = qt.tensor(I,H)
HI = qt.tensor(H,I)

def initial_state():
    return phi_p

def alice(x, y, state):
    if x == 1:
        state = qt.tensor(X, I) * state
    if y == 1:
        state = qt.tensor(Z, I) * state
    return state
def bob(state):
    state = CNOT21 * state
    state = qt.tensor(I,H) * state
    return state
def measure(state):
    p0 = abs(state.overlap(qt.tensor(zero, zero)))**2
    p1 = abs(state.overlap(qt.tensor(zero, one)))**2
    p2 = abs(state.overlap(qt.tensor(one, zero)))**2
    p3 = abs(state.overlap(qt.tensor(one, one)))**2
    return np.random.choice([0, 1, 2, 3], p=[p0, p1, p2, p3])


for x in [0, 1]:
    for y in [0, 1]:
        state = initial_state()
        state = alice(x, y, state)
        state = bob(state)
        result = measure(state)
        print(f"Alice sent ({x}{y}), Bob received ({result//2}{result%2})")


Alice sent (00), Bob received (00)
Alice sent (01), Bob received (01)
Alice sent (10), Bob received (10)
Alice sent (11), Bob received (11)


In [280]:
qt.tensor(I,I), qt.tensor(Z,I), qt.tensor(I,X), qt.tensor(Z,X)

(Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dia, isherm=True
 Qobj data =
 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]],
 Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=True
 Qobj data =
 [[ 1.  0.  0.  0.]
  [ 0.  1.  0.  0.]
  [ 0.  0. -1.  0.]
  [ 0.  0.  0. -1.]],
 Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=True
 Qobj data =
 [[0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]],
 Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=True
 Qobj data =
 [[ 0.  1.  0.  0.]
  [ 1.  0.  0.  0.]
  [ 0.  0.  0. -1.]
  [ 0.  0. -1.  0.]])

In [319]:
T = 1/np.sqrt(2) * qt.Qobj(np.array([
    [ 1,  0,  0,  1],
    [ 1,  0,  0, -1],
    [ 0, -1,  1,  0],
    [ 0,  1, -1,  0]]), dims=[[2,2],[2,2]])
# print(T)
print(psi_m.basis_expansion())
print((CNOT21 * psi_m).basis_expansion())
print((IH*CNOT21 * psi_m).basis_expansion())

(-0.70710678118655+0j) |10> + (0.70710678118655+0j) |01>
(0.70710678118655+0j) |11> + (-0.70710678118655+0j) |10>
(-1+0j) |11>


In [324]:
print(phi_m.basis_expansion())
print((CNOT21 * phi_m).basis_expansion())
print((IH*CNOT21 * phi_m).basis_expansion())

(-0.70710678118655+0j) |11> + (0.70710678118655+0j) |00>
(0.70710678118655+0j) |00> + (-0.70710678118655+0j) |01>
(1+0j) |01>
